In [1]:
from google.colab import files
uploaded = files.upload()

Saving Train_data.xlsx - Final.csv to Train_data.xlsx - Final.csv


In [4]:
import pandas as pd
import io
import warnings
import numpy as np
warnings.filterwarnings('ignore')

df = pd.read_csv(io.BytesIO(uploaded['Train_data.xlsx - Final.csv']))
df1 = df.copy()
df.head()

,Warehouse id,Region,SKU id,Apr-18,May-18,Jun-18,Jul-18,Aug-18,Sep-18,Oct-18,...,Aug-20,Sep-20,Oct-20,Nov-20,Dec-20,Jan-21,Feb-21,Mar-21,Apr-21,May-21
0,Wh-1,NORTH,SKU-1,27,13,30,14,22,8,8,...,12,8,8,8,8,8,14,16,8,12
1,Wh-1,NORTH,SKU-2,88,28,127,130,58,10,7,...,16,7,7,7,7,7,13,43,13,43
2,Wh-1,NORTH,SKU-3,6,6,6,6,6,6,6,...,8,6,6,6,6,6,6,8,26,6
3,Wh-1,NORTH,SKU-4,162,184,110,172,90,40,18,...,82,78,38,18,22,28,36,352,58,18
4,Wh-1,NORTH,SKU-5,171,402,627,615,300,417,141,...,468,216,75,6,30,66,141,699,183,24


In [5]:
#replace Wh-1 with 1 and SKU-1 with 1
df['SKU id']=pd.Series([int(id[4:]) for id in df["SKU id"].values])
df['Warehouse id']=pd.Series([int(id[3:]) for id in df["Warehouse id"].values])
df

,Warehouse id,Region,SKU id,Apr-18,May-18,Jun-18,Jul-18,Aug-18,Sep-18,Oct-18,...,Aug-20,Sep-20,Oct-20,Nov-20,Dec-20,Jan-21,Feb-21,Mar-21,Apr-21,May-21
0,1,NORTH,1,27,13,30,14,22,8,8,...,12,8,8,8,8,8,14,16,8,12
1,1,NORTH,2,88,28,127,130,58,10,7,...,16,7,7,7,7,7,13,43,13,43
2,1,NORTH,3,6,6,6,6,6,6,6,...,8,6,6,6,6,6,6,8,26,6
3,1,NORTH,4,162,184,110,172,90,40,18,...,82,78,38,18,22,28,36,352,58,18
4,1,NORTH,5,171,402,627,615,300,417,141,...,468,216,75,6,30,66,141,699,183,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,4,SOUTH,229,7,7,7,7,7,7,7,...,591,167,455,215,135,239,295,7,7,135
1035,4,SOUTH,230,49,25,41,13,21,53,33,...,9,9,21,133,9,33,9,103,49,9
1036,4,SOUTH,231,21,13,13,21,29,21,21,...,13,13,13,13,13,21,13,13,13,13
1037,4,SOUTH,232,15,13,13,11,9,11,36,...,9,9,17,13,11,36,21,17,9,9


In [6]:
wh = df1.groupby('Warehouse id')                              #separate dataframes for each warehouse
wh_data1 = wh.get_group('Wh-1')
wh_data2 = wh.get_group('Wh-2')
wh_data3 = wh.get_group('Wh-3')
wh_data4 = wh.get_group('Wh-4')

In [ ]:
!pip install statsmodels --upgrade

In [7]:
#AUTOREGRESSOR
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_absolute_percentage_error
import sys

     |████████████████████████████████| 9.8 MB 4.2 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [35]:
#WAREHOUSE 1
#For each SKU id, applying AR model will give best possible predictions for some value of lag
#So, for each SKU id in each warehouse, we iterate over the allowed values of lag (here, 1 to 17) to find lag value giving best accuracy
#Finally, we store the min_mape and (predicted,expected) for the lag value giving best accuracy for a SKU id.

print('Total elements in warehouse 1 :',len(wh_data1))
for i in range(1,3):
	min_mape_list, pred_exp_list  = [], [], []
	for row in range(len(wh_data1)):

		min_mape = sys.float_info.max
		pred_exp = ()
		for lag in range(1,18):
			x=wh_data1.iloc[row,3:].values
			train,test = x[:len(x)-i],x[len(x)-i:]								#Taking the last i elements of timeseries as test set
			model = AutoReg(train, lags=lag)
			model_fit = model.fit()
			predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
			mape = mean_absolute_percentage_error(test, predictions)
			if mape < min_mape:
				min_mape = mape
				pred_exp = predictions,test

		min_mape_list.append(min_mape)
		pred_exp_list.append(pred_exp)
	
# for i in range(len(pred_exp_list)):
# 	if min_mape_list[i]>5:
# 		print('Min. MAPE :',min_mape_list[i])
# 		print('Predictions, Expectations :')
# 		for j in range(len(pred_exp_list[i][0])):
# 			print(pred_exp_list[i][0][j],pred_exp_list[i][1][j])
	 
	print('Size of test data :',i)
	min_mape_list_5 = [i for i in range(len(min_mape_list)) if min_mape_list[i]>5]
	print('No. of elements in warehouse 1 with MAPE > 5 :',len(min_mape_list_5))
	print('Average MAPE for these elements :%.3f' % sum(min_mape_list_5)/len(min_mape_list_5))

Total elements in warehouse 1 : 233
Size of test data : 1
No. of elements in warehouse 1 with MAPE > 5 : 19
Average MAPE for these elements : 113.57894736842105
Size of test data : 2
No. of elements in warehouse 1 with MAPE > 5 : 44
Average MAPE for these elements : 111.70454545454545


In [36]:
#WAREHOUSE 2
print('Total elements in warehouse 2 :',len(wh_data2))
for i in range(1,3):
	min_mape_list, pred_exp_list  = [], [], []
	for row in range(len(wh_data2)):

		min_mape = sys.float_info.max
		pred_exp = ()
		for lag in range(1,18):
			x=wh_data2.iloc[row,3:].values
			train,test = x[:len(x)-i],x[len(x)-i:]								#Taking the last i elements of timeseries as test set
			model = AutoReg(train, lags=lag)
			model_fit = model.fit()
			predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
			mape = mean_absolute_percentage_error(test, predictions)
			if mape < min_mape:
				min_mape = mape
				pred_exp = predictions,test

		min_mape_list.append(min_mape)
		pred_exp_list.append(pred_exp)
	
# for i in range(len(pred_exp_list)):
# 	if min_mape_list[i]>5:
# 		print('Min. MAPE :',min_mape_list[i])
# 		print('Predictions, Expectations :')
# 		for j in range(len(pred_exp_list[i][0])):
# 			print(pred_exp_list[i][0][j],pred_exp_list[i][1][j])
	 
	print('Size of test data :',i)
	min_mape_list_5 = [i for i in range(len(min_mape_list)) if min_mape_list[i]>5]
	print('No. of elements in warehouse 2 with MAPE > 5 :',len(min_mape_list_5))
	print('Average MAPE for these elements : %.3f' % sum(min_mape_list_5)/len(min_mape_list_5))

Total elements in warehouse 2 : 212
Size of test data : 1
No. of elements in warehouse 2 with MAPE > 5 : 17
Average MAPE for these elements : 106.6470588235294
Size of test data : 2
No. of elements in warehouse 2 with MAPE > 5 : 18
Average MAPE for these elements : 92.16666666666667


In [37]:
#WAREHOUSE 3
print('Total elements in warehouse 3 :',len(wh_data3))
for i in range(1,3):
	min_mape_list, pred_exp_list  = [], [], []
	for row in range(len(wh_data3)):

		min_mape = sys.float_info.max
		pred_exp = ()
		for lag in range(1,18):
			x=wh_data3.iloc[row,3:].values
			train,test = x[:len(x)-i],x[len(x)-i:]								#Taking the last i elements of timeseries as test set
			model = AutoReg(train, lags=lag)
			model_fit = model.fit()
			predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
			mape = mean_absolute_percentage_error(test, predictions)
			if mape < min_mape:
				min_mape = mape
				pred_exp = predictions,test

		min_mape_list.append(min_mape)
		pred_exp_list.append(pred_exp)
	
# for i in range(len(pred_exp_list)):
# 	if min_mape_list[i]>5:
# 		print('Min. MAPE :',min_mape_list[i])
# 		print('Predictions, Expectations :')
# 		for j in range(len(pred_exp_list[i][0])):
# 			print(pred_exp_list[i][0][j],pred_exp_list[i][1][j])
	 
	print('Size of test data :',i)
	min_mape_list_5 = [i for i in range(len(min_mape_list)) if min_mape_list[i]>5]
	print('No. of elements in warehouse 3 with MAPE > 5 :',len(min_mape_list_5))
	print('Average MAPE for these elements : %.3f' % sum(min_mape_list_5)/len(min_mape_list_5))

Total elements in warehouse 3 : 308
Size of test data : 1
No. of elements in warehouse 3 with MAPE > 5 : 4
Average MAPE for these elements : 100.75
Size of test data : 2
No. of elements in warehouse 3 with MAPE > 5 : 23
Average MAPE for these elements : 185.95652173913044


In [38]:
#WAREHOUSE 4
print('Total elements in warehouse 4 :',len(wh_data4))
for i in range(1,3):
	min_mape_list, pred_exp_list  = [], [], []
	for row in range(len(wh_data4)):

		min_mape = sys.float_info.max
		pred_exp = ()
		for lag in range(1,18):
			x=wh_data4.iloc[row,3:].values
			train,test = x[:len(x)-i],x[len(x)-i:]								#Taking the last i elements of timeseries as test set
			model = AutoReg(train, lags=lag)
			model_fit = model.fit()
			predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
			mape = mean_absolute_percentage_error(test, predictions)
			if mape < min_mape:
				min_mape = mape
				pred_exp = predictions,test

		min_mape_list.append(min_mape)
		pred_exp_list.append(pred_exp)
	
# for i in range(len(pred_exp_list)):
# 	if min_mape_list[i]>5:
# 		print('Min. MAPE :',min_mape_list[i])
# 		print('Predictions, Expectations :')
# 		for j in range(len(pred_exp_list[i][0])):
# 			print(pred_exp_list[i][0][j],pred_exp_list[i][1][j])
	 
	print('Size of test data :',i)
	min_mape_list_5 = [i for i in range(len(min_mape_list)) if min_mape_list[i]>5]
	print('No. of elements in warehouse 4 with MAPE > 5 :',len(min_mape_list_5))
	print('Average MAPE for these elements : %.3f' % sum(min_mape_list_5)/len(min_mape_list_5))

Total elements in warehouse 4 : 286
Size of test data : 1
No. of elements in warehouse 4 with MAPE > 5 : 38
Average MAPE for these elements : 126.76315789473684
Size of test data : 2
No. of elements in warehouse 4 with MAPE > 5 : 31
Average MAPE for these elements : 126.48387096774194
